<a href="https://colab.research.google.com/github/Fyfy1996/Natural_language_understanding/blob/master/Lab3_Subword_tokenizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Subword Tokenization

We will use a subword tokenization library from Huggingface.
Let's install the tokenizer and download the data.

In [0]:
!pip install tokenizers
# download some sample text 
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt

     |████████████████████████████████| 3.7MB 2.9MB/s 
--2020-02-13 16:22:40--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt’

botchan.txt         100%[===================>] 272.25K  --.-KB/s    in 0.05s   

2020-02-13 16:22:40 (5.71 MB/s) - ‘botchan.txt’ saved [278779/278779]

--2020-02-13 16:22:41--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.39.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.39.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 213450 (208K) [text/plain]
Saving to: ‘bert-base-cased-voca

# Import Tokenizer Packages

In [0]:
from tokenizers import (ByteLevelBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers


# First let's try out a pretrained WordPiece tokenizer used in BERT model.

You will notice the [CLS] and [SEP] tokens which are usually added at the beginning and end of a sentence in BERT model. You will learn more about BERT in lecture.



In [0]:
# Intialize tokenizer with BERT's BPE vocabulary
bert_tokenizer = BertWordPieceTokenizer("bert-base-cased-vocab.txt")

# Tokenize text
output = bert_tokenizer.encode("Hellooooooooo! How are you?")
print(output.tokens)

['[CLS]', 'hello', '##oo', '##oo', '##oo', '##oo', '!', 'how', 'are', 'you', '?', '[SEP]']


# Now, let's train your own custom WordPiece tokenizer.
We downloaded a file named `botchan.txt` at the beginning. Let's train a wordpiece model on this file.  

Recap: WordPiece expects the pretokenized word sequence.  
The `tokenizer` library has built-in pretokenizer for WordPiece model so you don't have to worry about it. But, if you want to, you can specify the type of builtin pretokenizer you want to use. 

You can check the list of built-in pretokenizers [here](https://github.com/huggingface/tokenizers/blob/master/bindings/python/src/pre_tokenizers.rs).

In [0]:
# First, initialize a new WordPiece tokenizer
my_tokenizer = BertWordPieceTokenizer()

# Customize pre-tokenization and decoding
my_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit(add_prefix_space=True)
my_tokenizer.decoder = decoders.WordPiece()

# And then train on some text file
my_tokenizer.train(["botchan.txt"])

# Now we can encode
my_tok_output = my_tokenizer.encode("Hellooooooooo! How are you?")
print("tokens: ", my_tok_output.tokens)
print("offsets:", my_tok_output.offsets)
print("getting a token from offsets:", my_tok_output.original_str[my_tok_output.offsets[0]])

tokens:  ['hello', '##oo', '##oo', '##oo', '##oo', '!', 'how', 'are', 'you', '?']
offsets: [(0, 5), (5, 7), (7, 9), (9, 11), (11, 13), (13, 14), (15, 18), (19, 22), (23, 26), (26, 27)]
getting a token from offsets: Hello


You can see that the output is different from pretrained Bert's WordPiece tokenizer.

# Now, let's train a SentencePiece Tokenizer.

SentencePieceBPETokenizer is already implemented for you so you don't have to define trainer.

In [0]:
# First, initialize a new SentencePiece tokenizer
my_sp_tokenizer = SentencePieceBPETokenizer()

# And then train
my_sp_tokenizer.train(["botchan.txt"], vocab_size=20000, min_frequency=2)

# Now we can encode
my_sp_tok_output = my_sp_tokenizer.encode("Hellooooooooo! How are you?")
print(my_sp_tok_output.tokens)


['▁Hell', 'oo', 'oo', 'oo', 'oo', 'o', '!', '▁How', '▁are', '▁you?']


# Similary, let's train a Byte-Level Tokenizer.

`Tokenizer`'s BBPE adds a special token 'Ġ' at the beginning of words.

In [0]:
# First, initialize a new Byte-level tokenizer
my_bbpe_tokenizer = ByteLevelBPETokenizer()

# And then train
my_bbpe_tokenizer.train(["botchan.txt"], vocab_size=20000, min_frequency=2)

# Now we can encode
my_bbpe_tok_output = my_bbpe_tokenizer.encode("Hellooooooooo! How are you?")
print(my_bbpe_tok_output.tokens)


['Hell', 'oo', 'oo', 'oo', 'oo', 'o', '!', 'ĠHow', 'Ġare', 'Ġyou', '?']


## How can we get back the tokens before BPE?

### For word piece:

In [0]:
"""
TODO: Your code here
"""
my_tokenizer.decode(my_tok_output.ids).replace(" ##","")

'hellooooooooo ! how are you ?'

For SentencePiece:

In [0]:
"""
TODO: Your code here
"""
my_sp_tokenizer.decode(my_sp_tok_output.ids)

'Hellooooooooo! How are you?'

For byte-level BPE:

In [0]:
"""
TODO: Your code here
"""

Tokens before BBPE:  Hellooooooooo! How are you?


As you can see, it's usually pretty straightforward to de-bpe the bpe tokens. Detokenizing the tokens to revert the pretokenization in WordPiece is a different story.

# Implement the original BPE algorithm on your own.

### Reference: 
Sennrich, Rico, Barry Haddow, and Alexandra Birch. "Neural Machine Translation of Rare Words with Subword Units." Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). Vol. 1. 2016. http://www.aclweb.org/anthology/P16-1162.

Recap: Original BPE expects pretokenized sequence of words as input.  


Unlike SentencePiece, it doesn't use a special chracter for space. Instead, it uses an end of word symbol, '</w>' in our case.

First, let's create a method that adds '</w>' at the end of each word and convert a word into character tokens with space in between.

"NLP" -> "N L P \</w>"

In [0]:
# toy training data
train_sents = ["what is the higher mountain ?", "higher than highest", "this is the widest river", "newest iphone is cool"]

In [0]:
import collections, re

def get_vocab(sentences):
    """
        This function appends </w> at the end of each word
        and add space between characters of the word
        and count the occurence of each word
    """
    vocab = collections.defaultdict(int)
    for sent in sentences:
        words = sent.strip().split()
        for word in words:
            vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab
tmp_vocab = get_vocab(train_sents)
print(tmp_vocab)

defaultdict(<class 'int'>, {'w h a t </w>': 1, 'i s </w>': 3, 't h e </w>': 2, 'h i g h e r </w>': 2, 'm o u n t a i n </w>': 1, '? </w>': 1, 't h a n </w>': 1, 'h i g h e s t </w>': 1, 't h i s </w>': 1, 'w i d e s t </w>': 1, 'r i v e r </w>': 1, 'n e w e s t </w>': 1, 'i p h o n e </w>': 1, 'c o o l </w>': 1})


We want to merge the most frequently occuring token pairs in our training data.
Let's write a function that will return the possible token pairs in our vocabulary and their frequency

In [0]:
def get_stats(vocab):
    """
       This function return the consecutive token pairs in our data and their count
    """
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        # add all the consecutive token pairs and their frequencies to counter `pairs`
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

tmp_pairs = get_stats(tmp_vocab)
tmp_pairs

defaultdict(int,
            {('?', '</w>'): 1,
             ('a', 'i'): 1,
             ('a', 'n'): 1,
             ('a', 't'): 1,
             ('c', 'o'): 1,
             ('d', 'e'): 1,
             ('e', '</w>'): 3,
             ('e', 'r'): 3,
             ('e', 's'): 3,
             ('e', 'w'): 1,
             ('g', 'h'): 3,
             ('h', 'a'): 2,
             ('h', 'e'): 5,
             ('h', 'i'): 4,
             ('h', 'o'): 1,
             ('i', 'd'): 1,
             ('i', 'g'): 3,
             ('i', 'n'): 1,
             ('i', 'p'): 1,
             ('i', 's'): 4,
             ('i', 'v'): 1,
             ('l', '</w>'): 1,
             ('m', 'o'): 1,
             ('n', '</w>'): 2,
             ('n', 'e'): 2,
             ('n', 't'): 1,
             ('o', 'l'): 1,
             ('o', 'n'): 1,
             ('o', 'o'): 1,
             ('o', 'u'): 1,
             ('p', 'h'): 1,
             ('r', '</w>'): 3,
             ('r', 'i'): 1,
             ('s', '</w>'): 4,
             

Now, let's write a function that merges the word pairs in vocabulary.
Refer to regex library for pattern matching: https://docs.python.org/3/library/re.html

In [0]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair)) # escape with space
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') # prepare regex query, 
                                                    # match only if both char in bigram is not already part of a token
                                                    # e.g: pair = (e, s), "l o w e s t" -> match, 
                                                    # "l o we s t" or "l o w e st" -> don't match
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

merge_vocab(('h', 'e'), tmp_vocab)

{'? </w>': 1,
 'c o o l </w>': 1,
 'h i g he r </w>': 2,
 'h i g he s t </w>': 1,
 'i p h o n e </w>': 1,
 'i s </w>': 3,
 'm o u n t a i n </w>': 1,
 'n e w e s t </w>': 1,
 'r i v e r </w>': 1,
 't h a n </w>': 1,
 't h i s </w>': 1,
 't he </w>': 2,
 'w h a t </w>': 1,
 'w i d e s t </w>': 1}

Finally, the following function is to keep track of tokens in our vocabulary since the beginning.

In [0]:
def get_tokens(vocab, tokens=None):
    if tokens is None:
        tokens = []
    for word, freq in vocab.items():
        word_tokens = word.split()
        tokens.extend(word_tokens)
    return list(set(tokens))

# Training BPE

Based on the functions above, can you write the training algoritm?



In [0]:
train_vocab = get_vocab(train_sents)

tokens = get_tokens(train_vocab)
print("vocab before training: ", train_vocab) 
print("number of tokens before training: ", tokens) # All characters

vocab before training:  defaultdict(<class 'int'>, {'w h a t </w>': 1, 'i s </w>': 3, 't h e </w>': 2, 'h i g h e r </w>': 2, 'm o u n t a i n </w>': 1, '? </w>': 1, 't h a n </w>': 1, 'h i g h e s t </w>': 1, 't h i s </w>': 1, 'w i d e s t </w>': 1, 'r i v e r </w>': 1, 'n e w e s t </w>': 1, 'i p h o n e </w>': 1, 'c o o l </w>': 1})
number of tokens before training:  ['</w>', 'v', 'p', 't', 'a', 'c', 'u', 'n', 'i', 'e', 's', 'g', 'm', 'd', 'l', 'w', 'h', 'o', '?', 'r']


In [0]:
num_merges = 5
for i in range(num_merges):

    # Get the token pair with max frequency
    # best_pair = ??
    pairs = get_stats(train_vocab)
    if not pairs:
      break
    best_pair = max(pairs, key=pairs.get)
    # update train_vocab using the merged best pair 
    train_vocab = merge_vocab(best, train_vocab)
    # update tokens using train_vocab
    tokens = get_tokens(train_vocab, tokens)

    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best_pair))
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')
print("vocab after training: ", train_vocab)

Iter: 0
Best pair: ('t', '</w>')
Tokens: ['p', 't', 'c', 'u', 's', '</w>', 'v', 'i', 'm', 'l', 'r', 'a', 'd', 'g', 'w', 'h', '?', 'n', 'he', 'e', 'o']
Number of tokens: 21
Iter: 1
Best pair: ('t', '</w>')
Tokens: ['p', 't', 'c', 'u', 's', '</w>', 'v', 'i', 'm', 'l', 'r', 'a', 'd', 'g', 'w', 'h', '?', 'n', 'he', 'e', 'o']
Number of tokens: 21
Iter: 2
Best pair: ('t', '</w>')
Tokens: ['p', 't', 'c', 'u', 's', '</w>', 'v', 'i', 'm', 'l', 'r', 'a', 'd', 'g', 'w', 'h', '?', 'n', 'he', 'e', 'o']
Number of tokens: 21
Iter: 3
Best pair: ('t', '</w>')
Tokens: ['p', 't', 'c', 'u', 's', '</w>', 'v', 'i', 'm', 'l', 'r', 'a', 'd', 'g', 'w', 'h', '?', 'n', 'he', 'e', 'o']
Number of tokens: 21
Iter: 4
Best pair: ('t', '</w>')
Tokens: ['p', 't', 'c', 'u', 's', '</w>', 'v', 'i', 'm', 'l', 'r', 'a', 'd', 'g', 'w', 'h', '?', 'n', 'he', 'e', 'o']
Number of tokens: 21
vocab after training:  {'w h a t </w>': 1, 'i s </w>': 3, 't he </w>': 2, 'h i g he r </w>': 2, 'm o u n t a i n </w>': 1, '? </w>': 1, 't h

## Questions: 
- Why does BPE help?
- Why is byte-level BPE more compact than the char-level BPE?
- Can you have unknown tokens if you use WordPiece, SentencePiece, or byte-level BPE?
- Imagine you are building a multilingual NLP system (very different languages with different characters), what kind of tokenization method would you use?
Byte Level
